In [1]:
import os
from os import listdir
from os.path import isfile,join
from datetime import datetime,timedelta
import requests
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from math import sqrt 
from collections import Counter
from datetime import date, datetime, timedelta
import time

In [2]:
# I. Automatiser le téléchargement des fichiers de données
# But : Créer un script qui télécharge les fichiers au fil des jours et 
#       évite de télécharger à nouveau les fichiers déjà téléchargés
#       (Fichiers publiés chaque jour)

# Inconvénient : nécessite de créer un script de job en local pour faire tourner cahque jour  
#               Si un fichier n'est pas récupérer le jour même, il ne peut pas être récupérer le jour suivant

## Pour info : Chemin du répertoire d'enregistrement des répertoires (identique au notebook) 
working_directory = os.getcwd()
print(f"Chemin d'accès au répertoire : {working_directory}")

# Création d'un dossier pour stocker les fichiers téléchargés
os.makedirs("sauvegarde_log",exist_ok=True)
os.makedirs("sauvegarde_degra",exist_ok=True)

Chemin d'accès au répertoire : C:\Users\truel


In [3]:
# Créer une fonction pour obtenir la date du jour
def get_today():
    today = datetime.today()
    return today.strftime("%Y-%m-%d")

# Récupérer année/mois/jour actuel
current_year = datetime.now().year
current_month = datetime.now().month
current_day = datetime.now().day

# Utiliser la fonction today
today = get_today()


# Construction de l'url de téléchargement en fonction du mois et du jour
for year in range(current_year,current_year+1):
    for month in range(current_month,current_month+1):
        for day in range(current_day,current_day+1):
            if year == current_year and month == current_month and day > current_day: # Ignorer les fichiers supérieurs à la date d'aujourd'hui
                continue
            if f"{year}-{month:02d}-{day:02d}" < today:   # Ignorer les fichiers déjà téléchargés
                continue

            url_log_vol = f"http://sc-e.fr/docs/logs_vols_2024-{month:02d}-{day:02d}.csv"
            file_log = f"sauvegarde_log/logs_vols_2024-{month:02d}-{day:02d}.csv"

            url_degrad = f"http://sc-e.fr/docs/degradations_2024-{month:02d}-{day:02d}.csv"
            file_degrad = f"sauvegarde_degra/degradations_2024-{month:02d}-{day:02d}.csv"

# Télécharger le fichier
            print(f"Téléchargement de {file_log}")
            print(f"Téléchargement de {file_degrad}")

            response = requests.get(url_log_vol)
            response = requests.get(url_degrad)

# Si réponse 200, on réalise le téléchargement
            if response.status_code == 200:
                with open(file_log,"wb") as file_log_vol:
                    file_log_vol.write(response.content)
                
                with open(file_degrad,"wb") as file_degradation:
                    file_degradation.write(response.content)
            else : 
                print(f"Téléchargement impossible de {file_log}. Code statut {response.status_code}")
                print(f"Téléchargement impossible de {file_degrad}. Code statut {response.status_code}")
print("Téléchargement terminé")
        

Téléchargement de sauvegarde_log/logs_vols_2024-06-11.csv
Téléchargement de sauvegarde_degra/degradations_2024-06-11.csv
Téléchargement terminé


In [4]:
# Vérifier le nombre de fichiers téléchargés dans sauvegarde_log
log_path = "./sauvegarde_log"
listfile_log = [file for file in listdir(log_path) if isfile(join(log_path,file))]
print(f" Nb fichier log_vol",len(listfile_log))

degra_path = "./sauvegarde_degra"
listfile_degra = [file for file in listdir(degra_path) if isfile(join(degra_path,file))]
print(f" Nb fichier degradation",len(listfile_degra))

 Nb fichier log_vol 1
 Nb fichier degradation 1


In [14]:
# Vérifier l'ouverture d'un fichier log_vol
link_log = '/Users/nadiaaitmada/Desktop/Wild/projet3/sauvegarde_log/logs_vols_2024-06-07.csv'
df_log = pd.read_csv(link_log)
df_log.head()

,ref_vol,aero_linked,jour_vol,time_en_air,sensor_data,etat_voyant
0,V06683852,B737_0940,2024-06-05,0.6,"{'temp': '17.8°C', 'pressure': '904.2 hPa', 'v...",0
1,V03551025,E175_6879,2024-06-05,5.2,"{'temp': '-1.3°C', 'pressure': '1029.6 hPa', '...",0
2,V04512442,A320_2562,2024-06-05,7.9,"{'temp': '16.4°C', 'pressure': '1071.7 hPa', '...",0
3,V06831692,A320_1884,2024-06-05,7.4,"{'temp': '-24.3°C', 'pressure': '905.9 hPa', '...",1
4,V05264659,A321_3033,2024-06-05,3.6,"{'temp': '5.3°C', 'pressure': '1053.8 hPa', 'v...",0


In [15]:
# Vérifier l'ouverture d'un fichier degradation
link_degra = '/Users/nadiaaitmada/Desktop/Wild/projet3/sauvegarde_degra/degradations_2024-06-06.csv'
df_degra = pd.read_csv(link_degra)
df_degra.head()

,ref_deg,linked_aero,compo_concerned,usure_nouvelle,measure_day,need_replacement
0,D003661,E170_6353,REAE170-E170_6353-0,54.208480,2024-06-04,False
1,D009402,E170_6353,REAE170-E170_6353-1,51.052144,2024-06-04,False
2,D000812,E170_6353,SYSE170-E170_6353-2,17.457904,2024-06-04,False
3,D006665,E170_6353,ORDE170-E170_6353-3,7.373504,2024-06-04,False
4,D001163,E170_6353,AUTE170-E170_6353-4,43.057968,2024-06-04,False
